Slaps on coverpage and scratchwork page. Automatically removes the answerkey page

In [3]:
from pdfrw import PdfReader, PdfWriter
import os.path
testname='2018-final_v'###################CHANGE THIS######################
joinfilepath='C:/Users/souri/Dropbox/MiscTeachingStuff/pre_n_post/'


for n in range(5):
    inpfn=testname+str(n)+'.pdf'
    writer = PdfWriter()
    mcp=joinfilepath+'FinalCoverPage'+str(n)+'.pdf'###################FINAL EXAM
    #mcp=joinfilepath+'MidtermCoverPage'+str(n)+'.pdf'
    op_pg=joinfilepath+'OptionalPage.pdf'###################FINAL EXAM
    sp=joinfilepath+'scratchwork.pdf'
    if os.path.exists(inpfn) and os.path.exists(mcp) and os.path.exists(sp):
        writer.addpages(PdfReader(mcp).pages)
        writer.addpages(PdfReader(inpfn).pages[:-1])
        writer.addpages(PdfReader(op_pg).pages)
        writer.addpages(PdfReader(sp).pages)
        opfn='Test_v'+str(n)+'.pdf'
        print("Writing "+opfn)
        writer.write(opfn)
    
    

Writing Test_v0.pdf
Writing Test_v1.pdf
Writing Test_v2.pdf
Writing Test_v3.pdf


The following script transfers MP grades and test grades to the (downloaded) BS Gradebook

Steps:

Download MP as MP.csv. 

Calculate the scores in MP and put them in a column titled "MP Score". Make sure there is a heading row in MP.

Make sure All_processed.csv is ready (All_processed is the output of Grader)

Make sure Brightspace gradebook is downloaded as BSGrades.csv, and "out of" for tests are correct. 

The following code will create 3 columns extra columns in BSGrades for MPScores, test scores and test missed.

Then it will copy them over to the correct columns. Pay attention to the TWO #### below. 

Correct 'Test # Questions Missed Text Grade ' and 

Correct 'Midterm # Points Grade Numeric MaxPoints:40 Weight:33.333333333 Category:Midterms CategoryWeight:60>'
 

In [1]:
import pandas as pd

In [3]:
import pandas as pd
df=pd.read_csv('BSGrades.csv')

df_mp=pd.read_csv('MP.csv')
df_mp=df_mp.rename(columns={'Student ID': 'Serial Number Text Grade <Text>'})

df = pd.merge(df,df_mp[['Serial Number Text Grade <Text>','MP Score']],on='Serial Number Text Grade <Text>', how='left')

df_all=pd.read_csv('All_processed.csv')
df_all=df_all.rename(columns={'Srl No': 'Serial Number Text Grade <Text>'})

df = pd.merge(df,df_all[['Serial Number Text Grade <Text>','Score']],on='Serial Number Text Grade <Text>', how='left')
#df = pd.merge(df,df_all[['Serial Number Text Grade <Text>','Missed']],on='Serial Number Text Grade <Text>', how='left')

df['Mastering Physics Points Grade <Numeric MaxPoints:100 Weight:14.75>']=df['MP Score']

##### CHECK THIS CHECK THIS MAKE SURE YOU ARE DOING THE CORRECT TEST
df['Final Points Grade <Numeric MaxPoints:72 Weight:25>']=df['Score']

##### CHECK THIS CHECK THIS MAKE SURE YOU ARE DOING THE CORRECT TEST
#df['Test 3 Questions Missed Text Grade <Text>']=df['Missed']

columns = ['MP Score', 'Score']
df.drop(columns, inplace=True, axis=1)

df.to_csv('BSGrades.csv', index=False)

The following script calculates letter grades from the numerical grades, and also applies the reward for good dropped midterm. 

Create the final file as BSGrades_final.csv

Check the filenames to be renamed, particularly "Midterm1"

In [47]:
def lg(num_grade):
    if num_grade<59.5:
        return 'F'
    elif num_grade>=59.5 and num_grade<62.5:
        return 'D-'
    elif num_grade>=62.5 and num_grade<66.5:
        return 'D'
    elif num_grade>=66.5 and num_grade<69.5:
        return 'D+'
    elif num_grade>=69.5 and num_grade<72.5:
        return 'C-'
    elif num_grade>=72.5 and num_grade<76.5:
        return 'C'
    elif num_grade>=76.5 and num_grade<79.5:
        return 'C+'
    elif num_grade>=79.5 and num_grade<82.5:
        return 'B-'
    elif num_grade>=82.5 and num_grade<86.5:
        return 'B'
    elif num_grade>=86.5 and num_grade<89.5:
        return 'B+'
    elif num_grade>=89.5 and num_grade<92.5:
        return 'A-'
    else:
        return 'A'

def f(x):
    pointmax=40;
    if min(x[0]/pointmax,x[1]/pointmax,x[2]/pointmax)>=0.8:
        return x[3]+.5
    else:
        return x[3]

In [58]:
import pandas as pd
df=pd.read_csv('BSGrades_final.csv')
columns = ['Calculated Final Grade Denominator', 'Calculated Final Grade Denominator','Adjusted Final Grade Numerator','Adjusted Final Grade Denominator','End-of-Line Indicator']
df.drop(columns, inplace=True, axis=1)
########CHECK THIS!!!!! Fix column names to be renamed################
df.rename(columns={'Calculated Final Grade Numerator': 'Numerical Grade', 
                  'Midterm1 Points Grade <Numeric MaxPoints:40 Weight:33.333333333 Category:Midterms CategoryWeight:60>':'Midterm 1',
                  'Midterm 2 Points Grade <Numeric MaxPoints:40 Weight:33.333333333 Category:Midterms CategoryWeight:60>':'Midterm 2',
                  'Midterm 3 Points Grade <Numeric MaxPoints:40 Weight:33.333333333 Category:Midterms CategoryWeight:60>':'Midterm 3'}, inplace=True)
df["Letter"] = ""

df['Numerical Grade'] = df[['Midterm 1','Midterm 2','Midterm 3','Numerical Grade']].apply(f, axis=1)
df['Letter'] = df['Numerical Grade'].map(lg)
df.to_csv('OverallGrades.csv', index=False)